In [1]:
# Core
import numpy as np
import pandas as pd
import dask.dataframe as dd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from horizons import make_sim_horizons
from rebound_utils import make_sim_planets, integrate_numpy, integrate_df
from planets_test import get_integration_diff
from db_config import db_engine
from db_utils import sp2df, get_columns, df2csv, csv2db_stage, csv2db, csvs2db

In [3]:
# Typing
from typing import Tuple

In [4]:
# Make sim with planets
epoch = 59000
sim_epoch = make_sim_planets(epoch=epoch)

Added bodies to simulation.


In [5]:
# Date range to process
mjd0: int = epoch-100
mjd1: int = epoch+100

In [6]:
# Shared time_step and save_step
time_step: int = 1

# Flags for building simulation archive
save_elements: bool = True
progbar: bool = True

In [7]:
# Save simulation as Numpy arrays
# body_ids, body_names, epochs, q, v, elts = integrate_numpy(sim_epoch=sim_epoch, mjd0=mjd0, mjd1=mjd1, time_step=time_step, save_elements=save_elements, progbar=True)

In [8]:
df = integrate_df(sim_epoch=sim_epoch, mjd0=mjd0, mjd1=mjd1, time_step=time_step, save_elements=save_elements, progbar=True)

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
df[0:10]

,TimeID,BodyID,MJD,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,84816000,10,58900.0,-0.004221,0.007326,0.000035,-0.000008,-0.000002,2.283442e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84816000,1,58900.0,-0.217196,0.256225,0.039910,-0.027079,-0.017177,1.080142e-03,0.387097,0.205652,0.122239,0.843105,0.509335,0.927129,0.627633
2,84816000,2,58900.0,0.047415,0.725334,0.006907,-0.020251,0.001344,1.186871e-03,0.723328,0.006747,0.059246,1.337354,0.957278,-0.795348,-0.785747
3,84816000,399,58900.0,-0.873491,0.478566,0.000017,-0.008494,-0.015196,1.518513e-06,1.000768,0.017385,0.000077,2.886969,-1.065969,0.823827,0.798538
4,84816000,301,58900.0,-0.872089,0.476317,-0.000083,-0.007999,-0.014910,-4.543185e-05,0.002577,0.046343,0.089001,1.691851,1.293892,-3.997685,2.213882
5,84816000,4,58900.0,-0.784999,-1.305526,-0.008318,0.012547,-0.005955,-4.325253e-04,1.523691,0.093487,0.032254,0.863961,-1.280166,-1.691031,-1.504104
6,84816000,5,58900.0,0.898803,-5.122727,0.001138,0.007340,0.001664,-1.711022e-04,5.203469,0.048696,0.022752,1.754349,-1.509899,4.642178,-1.543644
7,84816000,6,58900.0,4.039565,-9.170298,-0.001362,0.004796,0.002233,-2.297898e-04,9.580375,0.051093,0.043393,1.982610,-0.388304,-2.750078,-2.709661
8,84816000,7,58900.0,16.104182,11.540741,-0.165769,-0.002320,0.003014,4.124782e-05,19.178276,0.046451,0.013448,1.293048,1.723148,-2.394865,-2.330122
9,84816000,8,58900.0,29.271555,-6.202268,-0.546868,0.000630,0.003090,-7.813982e-05,30.199682,0.010196,0.030895,2.299890,-1.986125,-0.522547,-0.512437


In [10]:
mjd0 = 55350
mjd1 = 62650
dfd = get_integration_diff(BodyCollectionCD='P', mjd0=mjd0, mjd1=mjd1, by_date=False)

In [11]:
# dfd[0:12]

In [12]:
chunksize=1000
ddf = dd.from_pandas(df, chunksize=chunksize, name='Integration')

In [13]:
ddf

,TimeID,BodyID,MJD,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
npartitions=3,,,,,,,,,,,,,,,,
0,int32,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1000,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
# Columns for state vectors and orbital element frames
cols_vec = ['TimeID', 'BodyID', 'MJD', 'qx', 'qy', 'qz', 'vx', 'vy', 'vz']
cols_elt = ['TimeID', 'BodyID', 'MJD', 'a', 'e', 'inc', 'Omega', 'omega', 'f', 'M']

elt_col_map = {
    'Omega':'Omega_node', 
    'omega': 'omega_peri',
}

# DataFrame with the state vectors
df_vec = df[cols_vec]
    
mask = (df.BodyID != 10)
df_elt = df[mask][cols_elt]
df_elt.rename(columns=elt_col_map, inplace=True)

In [15]:
# columns = get_columns(schema='KS', table='StateVectors')
# fname_csv = '~/Harvard/kepler-sieve/data/df2db/Integration_Planets.csv'
# chunksize = 1000
# fnames_csv = df2csv(df=df, fname_csv=fname_csv, columns=columns, chunksize=chunksize)

In [16]:
# pd.read_csv(fnames_csv[0])

In [17]:
# i = 0
# with db_engine.connect() as conn:    
#     csv2db_stage(schema='KS', table='Integration_Planets', columns=columns, fname_csv=fnames_csv[i], i=0, conn=conn)

In [18]:
# with db_engine.connect() as conn:    
#     csv2db(schema='KS', table='Integration_Planets', columns=columns, i=0, conn=conn)

In [19]:
# csvs2db(schema='KS', table='Integration_Planets', fnames_csv=fnames_csv, columns=columns, progbar=True)

In [20]:
columns_elt = get_columns(schema='KS', table='OrbitalElements')
fname_csv = '~/Harvard/kepler-sieve/data/df2db/OrbitalElements.csv'
chunksize = 1000

In [21]:
fnames_csv = df2csv(df=df_elt, fname_csv=fname_csv, columns=columns_elt, chunksize=chunksize)
fnames_csv

['/home/michael/Harvard/kepler-sieve/data/df2db/OrbitalElements-chunk-0.csv',
 '/home/michael/Harvard/kepler-sieve/data/df2db/OrbitalElements-chunk-1.csv',
 '/home/michael/Harvard/kepler-sieve/data/df2db/OrbitalElements-chunk-2.csv']

In [22]:
csvs2db(schema='KS', table='OrbitalElements', fnames_csv=fnames_csv, columns=columns_elt, progbar=True)

  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
import dask.dataframe

In [26]:
dask.dataframe.from_pandas

<function dask.dataframe.io.io.from_pandas(data, npartitions=None, chunksize=None, sort=True, name=None)>